## 🧠 1. User Query

In [11]:
query = "Is it charge any international charges?"

## ✅ Important Retriever Methods to Know

Here's what you’ll mostly use:

| Method | Description |
|--------|-------------|
| `get_relevant_documents(query)` | Main method — gets the top `k` chunks that are semantically similar to your query. |
| `as_retriever(**kwargs)` | Converts your vectorstore (like FAISS) into a retriever. You can pass search settings here like `k`. |
| `add_documents(docs)` | Add more docs to your index (useful for dynamic updates). |

---

### ➕ Retriever for:
- Adding **filters** (e.g., only return docs with `source='terms_and_conditions'`)
- Combining multiple retrievers (like metadata filtering + semantic search)
- Switching to a different backend (e.g., Chroma, Qdrant) without changing your code

---

## 🔄 2. Query Conversion

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [13]:
from langchain_openai import AzureOpenAIEmbeddings

In [14]:
embd = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
    model="text-embedding-3-small"
    )

In [15]:
query_vector = embd.embed_query(query)

## 📏 3. Vector Comparison

In [ ]:
from langchain import hub
from langchain_community.vectorstores import FAISS

In [17]:
new_index = FAISS.load_local("my_faiss_index", embd, allow_dangerous_deserialization=True)

## 🔝 4. Top-K Retrieval

In [18]:
retriever = new_index.as_retriever(search_kwargs={"k": 2})

## 📚 5. Data Retrieval

In [21]:
top_k_docs = retriever.get_relevant_documents(query)

In [22]:
for i, doc in enumerate(top_k_docs, 1):
    print(f"Doc {i}:", doc.page_content[:300], '\n')

Doc 1: and may be restricted to the original country of sale. Call 
charges and international shipping charges may apply, 
depending on the location. Subject to the full terms and 
detailed information on obtaining service available at 
apple.com/legal/warranty  and support.apple.com , if you 
submit a val 

Doc 2: Before using iPhone, review the iPhone User Guide  at  
support.apple.com/guide/iphone .
Safety and Handling
See “Safety, handling, and support” in the iPhone  
User Guide.
Exposure to Radio Frequency
On iPhone, go to Settings > General > Legal &  
Regulatory > RF Exposure. Or go to apple.com/ 
lega 



## 🆚 retriever.get_relevant_documents() vs faiss_index.similarity_search()

At a high level, **they both do the same thing**: find documents most relevant to a query.

But here’s the **real difference**:

| Feature | `retriever.get_relevant_documents()` | `faiss_index.similarity_search()` |
|--------|---------------------------------------|------------------------------------|
| **Interface** | Follows LangChain's `Retriever` interface | Direct call to the vector DB (FAISS) |
| **Use case** | Preferred in RAG pipelines and LangChain chains | Good for testing/debugging |
| **Flexibility** | You can plug in filters, hybrid search, etc. | Raw semantic search only |
| **Standardized** | Works across multiple vector stores | Tied to FAISS |
| **Returns** | List of `Document` objects | Same |

---


## 🏁 Summary

| Concept | Think of it as... |
|--------|--------------------|
| `similarity_search()` | A raw query to the FAISS engine |
| `get_relevant_documents()` | A standardized method to retrieve top docs for a query — the RAG-friendly way |
| `Retriever` | A wrapper that lets you swap out vector DBs and methods easily |

---